In [29]:
!git clone https://github.com/akshay5305/CNN-vs-VIT-for-Medical-Imaging.git

Cloning into 'CNN-vs-VIT-for-Medical-Imaging'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 20 (delta 2), reused 9 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 6.74 KiB | 6.74 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [30]:
%cd CNN-vs-VIT-for-Medical-Imaging


/content/CNN-vs-VIT-for-Medical-Imaging/CNN-vs-VIT-for-Medical-Imaging/CNN-vs-VIT-for-Medical-Imaging


In [31]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vuppalaadithyasairam/bone-fracture-detection-using-xrays")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/vuppalaadithyasairam/bone-fracture-detection-using-xrays/versions/1


In [32]:
import os

BASE_PATH = "/kaggle/input/bone-fracture-detection-using-xrays/archive (6)/val"

for root, dirs, files in os.walk(BASE_PATH):
    print(root, "->", dirs)
    break


/kaggle/input/bone-fracture-detection-using-xrays/archive (6)/val -> ['not fractured', 'fractured']


In [33]:
import tensorflow as tf


from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import SGD


from keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from utils import config as cfg
from utils.utils import return_classes


In [34]:
def return_ds(input_dir):
    train_ds=tf.keras.preprocessing.image_dataset_from_directory(input_dir,
    validation_split=0.2,
     subset='training',
     seed=42,
     image_size=(cfg.input_dim,cfg.input_dim),
     batch_size=16,
     label_mode='categorical',)

    val_ds=tf.keras.preprocessing.image_dataset_from_directory(
        input_dir,
        validation_split=0.2,
        subset='validation',
        seed=42,
        image_size=(cfg.input_dim,cfg.input_dim),
        batch_size=8,
        label_mode='categorical',)

    return train_ds, val_ds


In [35]:
def return_model(input_dim, nb_classes, freeze=False, head=None):
    if head=='xception' or head=='Xception':
        base_model=Xception(include_top=False,weights='imagenet',input_shape=(input_dim,input_dim,3))

    if head=='vg16' or head=='VGG16':
        base_model=VGG16(include_top=False, weights='imagenet',input_shape=(input_dim,input_dim,3))

    if head=='inceptionv3' or 'Inceptionv3' or 'InceptionV3':
        base_model=InceptionV3(include_top=False, weights='imagenet',input_shape=(input_dim,input_dim,3))

    if head=='densenet121':
        base_model=DenseNet121(include_top=False, weights='imagenet',input_shape=(input_dim,input_dim,3))

    if not head:
        print('Please choose pre trained model')

    print('Choose smodel is ', head)

    if freeze:
        for layer in base_model.layers:
            layer.trainable=False

    x=base_model.output
    x=GlobalAveragePooling2D()(x)

    predictions=Dense(nb_classes,activation='softmax')(x)
    model=Model(inputs=base_model.inputs, outputs=predictions)

    return model


In [ ]:
if __name__=='__main__':
    classes=return_classes('utils/annotation.txt')
    train_ds,val_ds=return_ds('/kaggle/input/bone-fracture-detection-using-xrays/archive (6)')
    model=return_model(cfg.input_dim,len(classes),head=cfg.head,freeze=True)


    model.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(learning_rate=cfg.lr),
    metrics=['accuracy']
)







    save_weights = ModelCheckpoint(filepath='models/xception_freeze_model_40_epoch.h5', monitor='val_accuracy',
                                    verbose=1, save_best_only=True, save_weights_only=False, mode='max')

    model.fit(train_ds, epochs=cfg.epochs, validation_data=val_ds, callbacks=[save_weights])



Found 9463 files belonging to 2 classes.
Using 7571 files for training.
Found 9463 files belonging to 2 classes.
Using 1892 files for validation.
Choose smodel is  xception
Epoch 1/40
474/474 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.8862 - loss: 166.3863
Epoch 1: val_accuracy improved from -inf to 0.06395, saving model to models/xception_freeze_model_40_epoch.h5


474/474 ━━━━━━━━━━━━━━━━━━━━ 75s 122ms/step - accuracy: 0.8862 - loss: 166.3060 - val_accuracy: 0.0640 - val_loss: 1939.0114
Epoch 2/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.9021 - loss: 117.5718
Epoch 2: val_accuracy improved from 0.06395 to 0.93605, saving model to models/xception_freeze_model_40_epoch.h5


474/474 ━━━━━━━━━━━━━━━━━━━━ 38s 80ms/step - accuracy: 0.9021 - loss: 117.4190 - val_accuracy: 0.9360 - val_loss: 148.0674
Epoch 3/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.9196 - loss: 59.6745
Epoch 3: val_accuracy improved from 0.93605 to 0.94027, saving model to models/xception_freeze_model_40_epoch.h5


474/474 ━━━━━━━━━━━━━━━━━━━━ 40s 83ms/step - accuracy: 0.9196 - loss: 59.6431 - val_accuracy: 0.9403 - val_loss: 14.6689
Epoch 4/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.9175 - loss: 58.5641
Epoch 4: val_accuracy did not improve from 0.94027
474/474 ━━━━━━━━━━━━━━━━━━━━ 37s 78ms/step - accuracy: 0.9175 - loss: 58.5089 - val_accuracy: 0.9376 - val_loss: 48.7594
Epoch 5/40
473/474 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.9305 - loss: 42.3750
Epoch 5: val_accuracy did not improve from 0.94027
474/474 ━━━━━━━━━━━━━━━━━━━━ 37s 78ms/step - accuracy: 0.9305 - loss: 42.3920 - val_accuracy: 0.9207 - val_loss: 19.1556
Epoch 6/40
372/474 ━━━━━━━━━━━━━━━━━━━━ 6s 60ms/step - accuracy: 0.9267 - loss: 53.1116